In [ ]:
import math, numpy as np, matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import rdDepictor

pt = Chem.GetPeriodicTable()                 # singleton periodic table

def valence_electrons(atom):
    return pt.GetNOuterElecs(atom.GetAtomicNum())

def lone_pairs_needed(atom):            # ❶  put it here
    ve   = valence_electrons(atom)
    fc   = atom.GetFormalCharge()
    bond_orders = sum(b.GetBondTypeAsDouble() for b in atom.GetBonds())
    nonbond_e   = ve - fc - int(bond_orders)     # correct formula
    return max(nonbond_e, 0) // 2                 # pairs only

def best_angles(used_dirs, n_needed):
    """
    Return `n_needed` angles (in radians) for lone-pair dots.

    • If there is **one dominant bond direction** (e.g. the N in a C≡N
      triple bond, or an O double-bonded to C), place the first pair
      180 ° opposite that bond.  For two lone pairs, put them ±60 °
      from that opposite direction so they end up ~120 ° apart.

    • If the atom has **two or more bonds**, fall back to the old
      “farthest-from-all-bonds” heuristic.
    """
    if n_needed == 0:
        return []

    # ---------- 1 bond direction  ----------
    if len(used_dirs) == 1:
        main = used_dirs[0]                       # bond angle in rad
        if n_needed == 1:                         # e.g. nitrile N
            return [(main + math.pi) % (2*math.pi)]

        # n_needed == 2  (e.g. carbonyl O in CO2)
        if n_needed == 2:
            base = (main + math.pi) % (2*math.pi) # opposite the bond
            offset = math.radians(60)             # ±60 °
            return [(base + offset) % (2*math.pi),
                    (base - offset) % (2*math.pi)]

        # n_needed ≥ 3 (rare: halides with LPs)
        base = (main + math.pi) % (2*math.pi)
        step = 2*math.pi / n_needed
        return [(base + k*step) % (2*math.pi) for k in range(n_needed)]

    # ---------- 2+ bond directions ----------
    cand  = np.linspace(0, 2*math.pi, 36, endpoint=False)
    score = [min(abs(math.atan2(math.sin(c-d), math.cos(c-d)))
                 for d in used_dirs)
             for c in cand]
    # pick the n_needed best-scoring directions
    return [c for c,_ in sorted(zip(cand, score), key=lambda x: -x[1])][:n_needed]

def draw_lewis_dots(smiles,
                    atom_font=24, dot_font=26,
                    bond_sep=0.22, lp_sep=0.28,
                    figscale=1.5, fig_dpi=150):
    """All-electron Lewis diagram with nicer scaling."""
    mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
    rdDepictor.Compute2DCoords(mol)
    conf = mol.GetConformer()

    xy = {i: np.array([conf.GetAtomPosition(i).x,
                       conf.GetAtomPosition(i).y])
          for i in range(mol.GetNumAtoms())}
    # normalise coordinates to ~unit box for consistent scaling
    coords = np.vstack(list(xy.values()))
    span   = coords.ptp(axis=0).max()
    for k in xy: xy[k] = (xy[k] - coords.mean(axis=0)) / span

    fig, ax = plt.subplots(figsize=(4*figscale, 4*figscale), dpi=fig_dpi)
    ax.axis('off'); ax.set_aspect('equal')

    # draw atoms
    for a in mol.GetAtoms():
        ax.text(*xy[a.GetIdx()], a.GetSymbol(), ha='center', va='center',
                fontsize=atom_font, fontweight='bold')

    # bonding-pair electrons (replace bond lines)
    for bond in mol.GetBonds():
        i, j   = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        A, B   = xy[i], xy[j]
        n_pair = int(bond.GetBondTypeAsDouble())        # 1,2,3
        for k in range(n_pair):
            t   = (k+1)/(n_pair+1)                      # 1/(n+1) ... n/(n+1)
            mid = A + (B - A) * t                      # ON the bond line
            # use a tiny perpendicular offset just to separate the two dots
            perp = np.array([-(B-A)[1], (B-A)[0]])
            perp /= np.linalg.norm(perp)
            delta = 0.05                               # much smaller than before
            for s in (+1, -1):
                pos = mid + s*perp*delta
                ax.text(*pos, '•', fontsize=dot_font, ha='center', va='center')

    # lone pairs
    for atom in mol.GetAtoms():
        idx = atom.GetIdx()            # 1️⃣  get the index first
        pos = xy[idx]
        
        # directions of bonds (angles in rad)
        bond_dirs = [math.atan2(*(xy[n.GetIdx()] - pos)[::-1])
                    for n in atom.GetNeighbors()]

        for ang in best_angles(bond_dirs, lone_pairs_needed(atom)):
            # radial vector of length lp_sep
            r_vec  = np.array([math.cos(ang), math.sin(ang)]) * lp_sep

            # mid-point of the pair (on that radial line)
            mid    = pos + r_vec

            # perpendicular (tangential) unit vector
            perp   = np.array([-r_vec[1], r_vec[0]])
            perp  /= np.linalg.norm(perp)

            # place the two dots ±along the tangent
            tang_sep = lp_sep * 0.25          # adjust 0.25 → wider/narrower pair
            for s in (+1, -1):
                dot = mid + s * perp * tang_sep
                ax.text(*dot, '•', fontsize=dot_font, ha='center', va='center')

    # remove extra whitespace tightly
    # 1) collect every text object's position
    xs, ys = [], []
    for txt in ax.texts:
        xs.append(txt.get_position()[0])
        ys.append(txt.get_position()[1])

    # 2) add a small cushion so nothing touches the edge
    pad = 0.08
    ax.set_xlim(min(xs) - pad, max(xs) + pad)
    ax.set_ylim(min(ys) - pad, max(ys) + pad)
    ax.set_aspect('equal')
    plt.show()

# ------- Demo molecules -------
draw_lewis_dots('O')        # water
draw_lewis_dots('O=C=O')        # CO₂   (4 pairs between C=O double bonds)
draw_lewis_dots('C(CO)N')       # ethanolamine
draw_lewis_dots('CC#N')         # acetonitrile, triple bond with 6 dots
draw_lewis_dots('CN(C)C')       # dimethylamine
draw_lewis_dots('c1ccncc1')     # pyridine
